<a href="https://colab.research.google.com/github/Ritu-95/Machine-Learning-An-Overview/blob/main/Part_of_Speech_Tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parts of Speech Tagging

The most basic and useful task while dealing with text based problems is to tokenize each word separately and label each word according to its most likely part of speech. This task is basically called as Part of Speech Tagging (POST).

In this lesson, we will use the Brown Corpus, an influential dataset that is used in many studies of POST. The Brown Corpus defined a tagset (specific collection of POS labels) that has been reused in many other annotated resources in English.

Recently, a different version of the tagset has been defined which is called the Universal Parts of Speech Tagset.

Let’s start with NLTK installation and downloading i.e. 
```
pip install nltk
nltk.download()
```

Content Courtesy: https://www.cs.bgu.ac.il/~elhadad/nlp19/nlp05.html#Using-Morphological-Clues

## The FreqDist NLTK Object: Counting Things

FreqDist (Frequency Distribution) is an object that counts occurrences of objects. It is defined in the nltk.probability module.

In [1]:
from nltk.probability import FreqDist
list = ['a', 'b', 'a']
fdist = FreqDist(list)
fdist

FreqDist({'a': 2, 'b': 1})

We can now use the FreqDist to explore the distribution of objects in the stream 'list':

In [2]:
fdist['a']  # We saw 'a' 2 times

2

In [3]:
fdist['b']  # We saw 'b' 1 time

1

In [4]:
fdist['c']  # We saw 'c' 0 times

0

In [5]:
fdist.max()      # 'a' is the most frequent sample seen

'a'

In [6]:
len(fdist)      # we observed 2 types of objects in the stream

2

In [7]:
fdist.keys()     # Return the types of the objects that were observed in the stream

dict_keys(['a', 'b'])

In [8]:
fdist.freq('a')  # 2/3 of the samples we saw were 'a' 

0.6666666666666666

In [9]:
fdist.N()        # How many samples did we count?

3

In [11]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

## Working on the Brown Corpus with NLTK

The tagged_sents version of the corpus is a list of sentences. Each sentence is a list of pairs (tuples) (word, tag). Similarly, one can access the corpus as a flat list of tagged words.

In [12]:
from nltk.corpus import brown
brown_news_tagged = brown.tagged_sents(tagset='universal')
brown_news_words = brown.tagged_words(tagset='universal')

In [14]:
import nltk
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

Let’s check: which word is the most common among the 100,000 words in this part of the Brown corpus? Which tag is the most common?

In [15]:
fdistw = FreqDist([w for (w, t) in brown_news_words])
fdistw.N()    # We saw 1,161,192 words in this section of the corpus

1161192

In [16]:
len(fdistw)  # How many distinct words are there|

56057

In [17]:
fdistw.max()  # What is the most frequent word?

'the'

In [18]:
fdistw['the']   # How often does 'the' occur in the corpus

62713

In [19]:
print('%5.2f%%' % (fdistw.freq('the') * 100))

 5.40%


We observe that the distribution of word tokens to word types is extremely unbalanced - a single word (the) accounts for over 6% of the word tokens. This is a general observation of linguistic data -- known as Zipf's Law -- few types are extremely frequent, and many types are extremely rare.

### Distinguishing Word Type and Work Token 

When distinguishing _word type_ and _word token_, we can decide to consider that strings that vary only because of case variants correspond to the same _word type_ - for example, the token _Book_ and _book_ correspond to the same word type _book_ (and so do _bOOk_ and _bOok_).

In [20]:
# Let us count the words without distinction of upper/lower case and the tags
fdistwl = FreqDist([w.lower() for (w, t) in brown_news_words])
fdistt = FreqDist([t for (w, t) in brown_news_words])

When we ignore case variants, there are only 49,815 word types instead of 62,713 when case differences are kept:

In [21]:
len(fdistwl)  # How many distinct words when we ignore case distinctions ('the' same as 'The')

49815

In [22]:
print('%5.2f%%' % (fdistt.freq('NOUN') * 100))

23.73%


## Perplexity of the POST task

The first question we address about the task of POS tagging is: how complex is it?

One way to quantify the complexity of a task is to measure its [perplexity](http://en.wikipedia.org/wiki/Perplexity). 
The intuitive meaning of perplexity is: when the tagger is considering which tag to associate to a word, how "confused" is it? That is, how many options does it have to choose from?

Obviously, this depends on the number of tags in the tagset. The [universal tagset](http://universaldependencies.org/u/pos/) includes 17 tags:

Tag	| Meaning	 | Examples
----|------------|----------
ADJ	| adjective	 | new, good, high, special, big, local
ADV	| adverb	 | really, already, still, early, now
CONJ| conjunction| and, or, but, if, while, although
DET	| determiner | the, a, some, most, every, no
X	| other, foreign words | dolce, ersatz, esprit, quo, maitre
NOUN | noun	     | year, home, costs, time, education
PROPN| proper noun | Alison, Africa, April, Washington
NUM	 | numeral	| twenty-four, fourth, 1991, 14:24
PRON | pronoun	| he, their, her, its, my, I, us
ADP  | adposition, preposition | on, of, at, with, by, into, under
AUX	 | auxiliary verb | has (done), is (doing), will (do), should (do), must (do), can (do)
INTJ | interjection | ah, bang, ha, whee, hmpf, oops
VERB | verb | is, has, get, do, make, see, run
PART | particle | possessive marker 's, negation 'not'
SCONJ | subordinating conjunction: complementizer, adverbial clause introducer | I believe 'that' he will come, if, while
SYM	| symbol | $, %, (C), +, *, /, =, :), john.doe@example.com


In the absence of any knowledge about words and tags, the perplexity of the task with a tagset of size 17 will be 17. We will see that adding knowledge will reduce the perplexity of the task.

Note that the decision on how to tag a word, without more information is ambiguous for multiple reasons:

- The same string can be understood as a noun or a verb (book).
- Some POS tags have a systematically ambiguous definition: a present participle can be used in progressive verb usages (I am going:VERB), but it can also be used in an adjectival position modifying a noun: (A striking:ADJ comparison). In other words, it is unclear in the definition itself of the tag whether the tag refers to a syntactic function or to a morphological property of the word.

## Measuring success: Accuracy, Training Dataset, Test Dataset

Let’s say, we develop a tagger, how to ensure if the tagger is successful or not, whether the decision made by the tagger is good or not?

The way to address these issues is to define a criterion for success, and to test the tagger on a large test dataset. Assume we have a large dataset of 1M words with their tags assigned manually. We first split the dataset into 2 parts: one part on which we will "learn" facts about the words and their tags (we call this the training dataset), and one part which we use to test the results of our tagger (we call this the test dataset).

It is critical NOT to test our tagger on the training dataset -- because we want to test whether the tagger is able to generalize from data it has seen and make decision on unseen data. (A "stupid" tagger would learn the exact data seen in the training dataset "by heart", and respond exactly as shown when asked on training data -- it would get a perfect score on the training data, but a poor score on any unseen data.)

This is one way to split the dataset into training and testing:

In [23]:
l = len(brown_news_tagged)
l

57340

In [24]:
# from nltk.corpus import brown
# brown_news_tagged = brown.tagged_sents(categories='news', tagset='universal')
# brown_news_words = brown.tagged_words(categories='news', tagset='universal')

brown_train = brown_news_tagged[5000:]
brown_test = brown_news_tagged[:5000]

from nltk.tag import untag
test_sent = untag(brown_test[0])
print("Tagged: ", brown_test[0])
print("Untagged: ", test_sent)

Tagged:  [('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')]
Untagged:  ['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']


To measure success, in this task, we will measure accuracy. The tagger object in NLTK includes a method called `evaluate` to measure the accuracy of a tagger on a given test set.


In [25]:
# A default tagger assigns the same tag to all words
from nltk import DefaultTagger
default_tagger = DefaultTagger('XYZ')
default_tagger.tag('This is a test'.split())

[('This', 'XYZ'), ('is', 'XYZ'), ('a', 'XYZ'), ('test', 'XYZ')]

Since 'NOUN' is the most frequent universal tag in the Brown corpus, we can use a tagger that assigns 'NOUN' 
to all words as a baseline.

In [26]:
default_tagger = DefaultTagger('NOUN')
print(default_tagger.tag(test_sent))

[('The', 'NOUN'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'NOUN'), ('Jury', 'NOUN'), ('said', 'NOUN'), ('Friday', 'NOUN'), ('an', 'NOUN'), ('investigation', 'NOUN'), ('of', 'NOUN'), ("Atlanta's", 'NOUN'), ('recent', 'NOUN'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'NOUN'), ('``', 'NOUN'), ('no', 'NOUN'), ('evidence', 'NOUN'), ("''", 'NOUN'), ('that', 'NOUN'), ('any', 'NOUN'), ('irregularities', 'NOUN'), ('took', 'NOUN'), ('place', 'NOUN'), ('.', 'NOUN')]


Using this baseline, we achieve a low accuracy:

In [27]:
print('Accuracy: %4.1f%%' % (100.0 * default_tagger.evaluate(brown_test)))

<ipython-input-27-af5fa0953a05>:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print('Accuracy: %4.1f%%' % (100.0 * default_tagger.evaluate(brown_test)))


Accuracy: 30.2%


Still, note that we improved the expected accuracy from picking one out of 17 answers with no knowledge, to picking one out of about 3 answers with very little knowledge (what is the most frequent tag in the dataset).

The computation of the accuracy compares the answers obtained by the tagger with that listed in the "gold standard" dataset. This is defined as follows in the NLTK code (the izip method in Python is defined in the itertools package, given two iterable collections, it returns an iterator over a collection of pairs - iterators are "consumed" by code such as "for ... in ...iterator...):

```python
 def accuracy(reference, test): 
       if len(reference) != len(test): 
           raise ValueError("Lists must have the same length.") 
       num_correct = 0 
       for x, y in izip(reference, test): 
           if x == y: 
               num_correct += 1 
       return float(num_correct) / len(reference)
```

## Sources of Knowledge to Improve Tagging Accuracy

Intuitively, the sources of knowledge that can help us decide what is the tag of a word include:
- A dictionary that lists the possible parts of speech for each word
- The context of the word in a sentence (neighboring words)
- The morphological form of the word (suffixes, prefixes)

We will now develop a sequence of taggers that use these knowledge sources, and combine them. 
The taggers we develop will implement the `nltk.tag.TaggerI` interface:

## Lookup Tagger: Using Dictionary Knowledge

Assume we have a dictionary that lists the possible tags for each word in English. Could we use this information to perform better tagging?

The intuition is that we would only assign to a word a tag that it can have in the dictionary. For example, if "box" can only be a Verb or a Noun, when we have to tag an instance of the word "box", we only choose between 2 options - and not between 17 options. Thus, dictionary knowledge will reduce the perplexity of the task.

There are 3 issues we must address to turn this into working code:

- Where do we get the dictionary?
- How do we choose between the various tags associated to a word in the dictionary? (For example, how do we choose between VERB and NOUN for "box").
- What do we do for words that do not appear in the dictionary?

The simple solutions we will test are the following - note that for each question, there exist other strategies that we will investigate later:

- Where do we get the dictionary: we will learn it from a sample dataset.
- How do we choose between the various tags associated to a word in the dictionary: we will choose the most likely tag as observed in the sample dataset.
- What do we do for words that do not appear in the dictionary: we will pass unknown words to a backoff tagger.

The `nltk.UnigramTagger` implements this overall strategy. It must be trained on a dataset, from which it builds a model of "unigrams". The following code shows how it is used:

In [28]:
# Prepare training and test datasets
# from nltk.corpus import brown
from nltk import UnigramTagger

# brown_news_tagged = brown.tagged_sents(categories='news', tagset='universal')
# brown_train = brown_news_tagged[100:]
# brown_test = brown_news_tagged[:100]

# Train the unigram model
unigram_tagger = UnigramTagger(brown_train)

# Test it on a single sentence
unigram_tagger.tag(untag(brown_test[0]))

[('The', 'DET'),
 ('Fulton', 'NOUN'),
 ('County', 'NOUN'),
 ('Grand', 'ADJ'),
 ('Jury', 'NOUN'),
 ('said', 'VERB'),
 ('Friday', 'NOUN'),
 ('an', 'DET'),
 ('investigation', 'NOUN'),
 ('of', 'ADP'),
 ("Atlanta's", None),
 ('recent', 'ADJ'),
 ('primary', 'ADJ'),
 ('election', 'NOUN'),
 ('produced', 'VERB'),
 ('``', '.'),
 ('no', 'DET'),
 ('evidence', 'NOUN'),
 ("''", '.'),
 ('that', 'ADP'),
 ('any', 'DET'),
 ('irregularities', 'NOUN'),
 ('took', 'VERB'),
 ('place', 'NOUN'),
 ('.', '.')]

Note that the unigram tagger leaves some words tagged as 'None' -- these are **unknown words**, words that were not observed in the training dataset.

How successful is this tagger?

In [29]:
print('Unigram tagger accuracy: %4.1f%%' % ( 100.0 * unigram_tagger.evaluate(brown_test)))

<ipython-input-29-2687c8d6c2f1>:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print('Unigram tagger accuracy: %4.1f%%' % ( 100.0 * unigram_tagger.evaluate(brown_test)))


Unigram tagger accuracy: 90.5%


90.5% is quite an improvement on the 31% of the default tagger. 
And this is without any backoff and without using morphological clues.

Is 90.5% a good level of accuracy? In fact it is not. It is accuracy per word. It means that on average, in every sentence of about 20 words, we will accumulate 2 errors. 2 errors in each sentence is a very high error rate. It makes it difficult to run another task on the output of such a tagger. Think how difficult the life of a parser would be if 2 words in every sentence are wrongly tagged. The problem is known as the **pipeline effect** -- when language processing tools are chained in a pipeline, error rates accumulate from module to module.

How much would a good backoff help? Let's try first to add the NN-default tagger as a backoff:

In [30]:
nn_tagger = DefaultTagger('NOUN')
ut2 = UnigramTagger(brown_train, backoff=nn_tagger)
print('Unigram tagger with backoff accuracy: %4.1f%%' % ( 100.0 * ut2.evaluate(brown_test)))

<ipython-input-30-147680b4cf96>:3: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print('Unigram tagger with backoff accuracy: %4.1f%%' % ( 100.0 * ut2.evaluate(brown_test)))


Unigram tagger with backoff accuracy: 94.5%


Adding a simple backoff (with accuracy of 31%) improved accuracy from 90.5% to 94.5%. 

One way to report this is in terms of **error reduction**: the error rate went down from 9.5% (100-90.5) to 5.5%. 
That's an **absolute error reduction** of 9.5-5.5 = 4.0%. 
Error reduction is generally reported as a percentage of the error: 100.0 * (4.0 / 9.5) = 42.1% relative error reduction. 

In other words, out of the words not tagged by the original model (with no backoff), 42.1% were corrected by the backoff.

What can we say about this error reduction? It is different from the accuracy of the backoff (42.1% vs 31%). 

One lesson to learn from this is that the **distribution of unknown words is significantly different from the distribution of all the words in the corpus**. 

## Summary

Parts of speech are classes of words that can be characterized by criteria of different types:

- Syntactic: 2 words of the same class can substitute each other in a sentence and leave the sentence syntactically acceptable
- Morphological: words of the same class are inflected in similar manner
- Semantic: words of the same class denote entities of similar semantic types (object, action, property, relation)


Tagsets of various granularities can be considered. We mentioned the standard Brown corpus tagset (about 60 tags for the complete tagset) and the reduced universal tagset (17 tags).
The key point of the approach we investigated is that it is data-driven: we attempt to solve the task by:
- Obtain sample data annotated manually: we used the Brown corpus
- Define a success metric: we used the definition of accuracy
- Measure the adequacy of a method by measuring its success
- Measure the complexity of the problem by using the notion of perplexity


The computational methods we developed are characterized by:
- We first define possible knowledge sources that can help us solve the task. Specifically, we investigated
dictionary,
morphological,
context as possible sources.
- We developed computational models that represent each of these knowledge sources in simple data structures (hash tables, frequency distributions, conditional frequency distributions).
- We tested simple machine learning methods: data is acquired by inspecting a training dataset, then evaluated by testing on a test dataset.
- We investigated one method to combine several systems into a combined system: backoff models.


This methodology will be further developed in the next chapters, as we will address more complex tasks (parsing, summarization) and use more sophisticated machine learning methods.

The task of Parts of Speech tagging is very well studied in English. The most efficient systems obtain accuracy rates of over 98% even on fine granularity tagsets - which is equivalent to the rate of success human beings obtain and the best agreement among human taggers generally obtained. The best systems use better machine learning algorithms (HMM, SVM) and treat unknown words (words not seen in training data) with more sophistication than what we have observed here.
